In [ ]:
from cloth_tools.dataset.download import get_latest_observation_start_url

server_url = "https://robotlab.ugent.be"

get_latest_observation_start_url(server_url) + "/image_left.png"

In [ ]:
from pathlib import Path
from cloth_tools.dataset.download import download_latest_observation

data_dir = Path("data")
dataset_dir = data_dir / "downloaded_dataset_0000"

observation_dir = download_latest_observation(dataset_dir, server_url)
observation_dir

In [ ]:
from cloth_tools.dataset.format import load_competition_observation

observation = load_competition_observation(observation_dir)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(observation.image_left)
plt.title("Left image")
plt.subplot(1, 2, 2)
plt.imshow(observation.image_right)
plt.title("Right image")
plt.show()

In [ ]:
from cloth_tools.dataset.bookkeeping import datetime_for_filename

datetime_for_filename()